# 责任链模式

将请求的发送方与处理请求的接收方解耦。

函数不需要直接调用别的函数，可以把请求发送给一个或者多个接受者组成的链条中。
首个接受者可以处理请求并停止，也可以把请求发送给下一个接收者。

In [6]:
import random
import string

random.seed(917) # Don't want random for regression tests

MOUSE, KEYPRESS, TIMER, TERMINATE = ("MOUSE", "KEYPRESS", "TIMER",
        "TERMINATE")


def next():
    kinds = ([MOUSE] * 7) + ([KEYPRESS] * 11) + ([TIMER] * 5) + [TERMINATE]
    kind = random.choice(kinds)
    if kind == MOUSE:
        return Event(kind, button=random.randint(1, 3),
                x=random.randint(0, 640), y=random.randint(0, 480))
    elif kind == KEYPRESS:
        return Event(kind, ctrl=random.randint(1, 7) == 1,
                shift=random.randint(1, 5) == 1,
                key=random.choice(string.ascii_lowercase))
    return Event(kind) # TIMER or TERMINATE


class Event:

    TimerId = 0

    def __init__(self, kind, **kwargs):
        assert kind in {MOUSE, KEYPRESS, TIMER, TERMINATE}
        self.kind = kind
        self.kwargs = kwargs
        if self.kind == TIMER:
            self.kwargs["id"] = Event.TimerId
            Event.TimerId += 1


    def __str__(self):
        if self.kind == MOUSE:
            return "Button {} ({}, {})".format(
                    self.kwargs.get("button", 1), self.kwargs.get("x", -1),
                    self.kwargs.get("y", -1))
        elif self.kind == KEYPRESS:
            return "Key {}{}{}".format(
                    "Ctrl+" if self.kwargs.get("ctrl", False) else "",
                    "Shift+" if self.kwargs.get("shift", False) else "",
                    self.kwargs.get("key", ""))
        elif self.kind == TIMER:
            return "Timer {}".format(self.kwargs.get("id", -1))
        elif self.kind == TERMINATE:
            return "Terminate"


In [10]:
import sys


def main():
    print("Handler Chain #1")
    handler1 = TimerHandler(KeyHandler(MouseHandler(NullHandler())))
    # Could pass None or nothing instead of the NullHandler
    while True:
        event = next()
        if event.kind == TERMINATE:
            break
        handler1.handle(event)

    print("\nHandler Chain #2 (debugging)")
    handler2 = DebugHandler(handler1)
    while True:
        event = next()
        if event.kind == TERMINATE:
            break
        handler2.handle(event)


class NullHandler:

    def __init__(self, successor=None):
        self.__successor = successor


    def handle(self, event):
        if self.__successor is not None:
            self.__successor.handle(event)


class DebugHandler(NullHandler):

    def __init__(self, successor=None, file=sys.stdout):
        super().__init__(successor)
        self.__file = file


    def handle(self, event):
        self.__file.write("*DEBUG*: {}\n".format(event))
        super().handle(event)


class MouseHandler(NullHandler):

    def handle(self, event):
        if event.kind == MOUSE:
            print("Click:   {}".format(event))
        else:
            super().handle(event)


class KeyHandler(NullHandler):

    def handle(self, event):
        if event.kind == KEYPRESS:
            print("Press:   {}".format(event))
        else:
            super().handle(event)


class TimerHandler(NullHandler):

    def handle(self, event):
        if event.kind == TIMER:
            print("Timeout: {}".format(event))
        else:
            super().handle(event)


if __name__ == "__main__":
    main()


Handler Chain #1
Timeout: Timer 0
Timeout: Timer 1
Click:   Button 2 (105, 144)
Timeout: Timer 2
Press:   Key r
Press:   Key Ctrl+s
Press:   Key z
Press:   Key Shift+j
Press:   Key Shift+j
Press:   Key y
Press:   Key x

Handler Chain #2 (debugging)
*DEBUG*: Key o
Press:   Key o
*DEBUG*: Button 2 (595, 66)
Click:   Button 2 (595, 66)
*DEBUG*: Key Ctrl+Shift+i
Press:   Key Ctrl+Shift+i
*DEBUG*: Key Ctrl+f
Press:   Key Ctrl+f
*DEBUG*: Key Shift+f
Press:   Key Shift+f
*DEBUG*: Timer 3
Timeout: Timer 3
*DEBUG*: Button 2 (311, 266)
Click:   Button 2 (311, 266)
*DEBUG*: Timer 4
Timeout: Timer 4
*DEBUG*: Key k
Press:   Key k
*DEBUG*: Button 1 (555, 220)
Click:   Button 1 (555, 220)
*DEBUG*: Button 3 (635, 138)
Click:   Button 3 (635, 138)
*DEBUG*: Button 3 (619, 295)
Click:   Button 3 (619, 295)
*DEBUG*: Button 3 (443, 34)
Click:   Button 3 (443, 34)
*DEBUG*: Timer 5
Timeout: Timer 5
*DEBUG*: Key Ctrl+t
Press:   Key Ctrl+t
*DEBUG*: Button 2 (533, 289)
Click:   Button 2 (533, 289)


In [13]:
#!/usr/bin/env python3
# Copyright © 2012-13 Qtrac Ltd. All rights reserved.
# This program or module is free software: you can redistribute it
# and/or modify it under the terms of the GNU General Public License as
# published by the Free Software Foundation, either version 3 of the
# License, or (at your option) any later version. It is provided for
# educational purposes and is distributed in the hope that it will be
# useful, but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU
# General Public License for more details.

import sys
from Qtrac import coroutine


def main():
    print("Handler Chain #1")
    pipeline = key_handler(mouse_handler(timer_handler()))
    while True:
        event = next()
        if event.kind == TERMINATE:
            break
        pipeline.send(event)

    print("\nHandler Chain #2 (debugging)")
    pipeline = debug_handler(pipeline)
    while True:
        event = next()
        if event.kind == TERMINATE:
            break
        pipeline.send(event)


@coroutine
def debug_handler(successor, file=sys.stdout):
    while True:
        event = (yield)
        file.write("*DEBUG*: {}\n".format(event))
        successor.send(event)


@coroutine
def mouse_handler(successor=None):
    while True:
        event = (yield)
        if event.kind == MOUSE:
            print("Click:   {}".format(event))
        elif successor is not None:
            successor.send(event)


@coroutine
def key_handler(successor=None):
    while True:
        event = (yield)
        if event.kind == KEYPRESS:
            print("Press:   {}".format(event))
        elif successor is not None:
            successor.send(event)


@coroutine
def timer_handler(successor=None):
    while True:
        event = (yield)
        if event.kind == TIMER:
            print("Timeout: {}".format(event))
        elif successor is not None:
            successor.send(event)


if __name__ == "__main__":
    main()


Handler Chain #1
Press:   Key q
Click:   Button 1 (320, 217)
Press:   Key x
Press:   Key g

Handler Chain #2 (debugging)
*DEBUG*: Timer 7
Timeout: Timer 7
*DEBUG*: Button 2 (304, 223)
Click:   Button 2 (304, 223)
*DEBUG*: Button 1 (8, 74)
Click:   Button 1 (8, 74)
*DEBUG*: Key m
Press:   Key m
*DEBUG*: Timer 8
Timeout: Timer 8
*DEBUG*: Key l
Press:   Key l
*DEBUG*: Key e
Press:   Key e
*DEBUG*: Key w
Press:   Key w
*DEBUG*: Timer 9
Timeout: Timer 9
